Importanto Módulos

In [1]:
import pandas as pd
import unidecode
import re
import matplotlib.pyplot as plt
import plotly.express as px

import os
import csv
import pandas as pd
import chardet

import folium
from folium.plugins import HeatMap
from folium.plugins import MousePosition
from folium import plugins

Importando Dados das Ocorrências Aeronáuticas

In [2]:
df_ocorr = pd.read_csv(r'..\data\Brazil Total Aeronautical Occurrences 2010 - 2021.csv')

In [3]:
# removendo colunas com valores constantes
lista_col_val_const = []
for coluna in df_ocorr.columns:

    if  df_ocorr[coluna].unique().size == 1:

        print(f"{coluna} : {df_ocorr[coluna].unique()}")
        lista_col_val_const.append(coluna)

df_ocorr.drop(columns=lista_col_val_const, inplace=True)

# removendo colunas com dados redundantes
lista_col_val_red=["recomendacao_destinatario_sigla", "aeronave_registro_categoria"]
df_ocorr.drop(columns=lista_col_val_red, inplace=True)

ocorrencia_pais : ['BRASIL']
investigacao_status : ['FINALIZADA']
divulgacao_relatorio_publicado : ['SIM']


In [4]:
# contando os dados faltantes nas colunas
print(df_ocorr.isnull().sum()[df_ocorr.isnull().sum() != 0])

# tratando os dados faltantes
df_ocorr["ocorrencia_latitude"] = df_ocorr["ocorrencia_latitude"].fillna("0")
df_ocorr["ocorrencia_longitude"] = df_ocorr["ocorrencia_longitude"].fillna("0")
df_ocorr["investigacao_aeronave_liberada"] = df_ocorr["investigacao_aeronave_liberada"].fillna("DESCONHECIDO")
df_ocorr["divulgacao_relatorio_numero"] = df_ocorr["divulgacao_relatorio_numero"].fillna("DESCONHECIDO")
df_ocorr["ocorrencia_tipo"] = df_ocorr["ocorrencia_tipo"].fillna("DESCONHECIDO")
df_ocorr["recomendacao_status"] = df_ocorr["recomendacao_status"].fillna("DESCONHECIDO")
df_ocorr["fator_condicionante"] = df_ocorr["fator_condicionante"].fillna("DESCONHECIDO")
df_ocorr["ocorrencia_tipo_categoria"] = df_ocorr["ocorrencia_tipo_categoria"].fillna("DESCONHECIDO")
df_ocorr["taxonomia_tipo_icao"] = df_ocorr["taxonomia_tipo_icao"].fillna("DESCONHECIDO")
df_ocorr["recomendacao_dia_feedback"] = df_ocorr["recomendacao_dia_feedback"].fillna("1900-01-01")
df_ocorr["recomendacao_status"] = df_ocorr["recomendacao_status"].fillna("DESCONHECIDA")
df_ocorr["aeronave_tipo_veiculo"] = df_ocorr["aeronave_tipo_veiculo"].fillna("DESCONHECIDO")
df_ocorr["aeronave_fabricante"] = df_ocorr["aeronave_fabricante"].fillna("DESCONHECIDO")
df_ocorr["aeronave_modelo"] = df_ocorr["aeronave_modelo"].fillna("DESCONHECIDO")
df_ocorr["aeronave_tipo_icao"] = df_ocorr["aeronave_tipo_icao"].fillna("DESCONHECIDO")
df_ocorr["aeronave_motor_tipo"] = df_ocorr["aeronave_motor_tipo"].fillna("DESCONHECIDO")
df_ocorr["aeronave_motor_quantidade"] = df_ocorr["aeronave_motor_quantidade"].fillna("DESCONHECIDO")
df_ocorr["aeronave_assentos"] = df_ocorr["aeronave_assentos"].fillna("0")
df_ocorr["aeronave_ano_fabricacao"] = df_ocorr["aeronave_ano_fabricacao"].fillna("0")
df_ocorr["aeronave_registro_segmento"] = df_ocorr["aeronave_registro_segmento"].fillna("DESCONHECIDO")
df_ocorr["aeronave_tipo_operacao"] = df_ocorr["aeronave_tipo_operacao"].fillna("DESCONHECIDO")
df_ocorr["aeronave_nivel_dano"] = df_ocorr["aeronave_nivel_dano"].fillna("DESCONHECIDO")
df_ocorr["fator_condicionante"] = df_ocorr["fator_condicionante"].fillna("DESCONHECIDO")

ocorrencia_latitude                670
ocorrencia_longitude               670
investigacao_aeronave_liberada    1648
divulgacao_relatorio_numero         39
ocorrencia_tipo                      4
ocorrencia_tipo_categoria            4
taxonomia_tipo_icao                  4
recomendacao_dia_feedback         1123
recomendacao_status                106
aeronave_tipo_veiculo               44
aeronave_fabricante                107
aeronave_modelo                     75
aeronave_tipo_icao                  94
aeronave_motor_tipo                 94
aeronave_motor_quantidade           21
aeronave_assentos                   50
aeronave_ano_fabricacao             47
aeronave_registro_segmento         183
aeronave_tipo_operacao              76
aeronave_nivel_dano                 12
fator_condicionante                802
dtype: int64


In [5]:
# filtrando colunas de interesse
df_ocorr = df_ocorr.drop([
    'investigacao_aeronave_liberada',
    'divulgacao_relatorio_numero',
    'divulgacao_dia_publicacao',
    'recomendacao_dia_assinatura',
    'recomendacao_dia_encaminhamento', 
    'recomendacao_dia_feedback',
    'recomendacao_numero',
    'recomendacao_conteudo',
    'aeronave_matricula',
    'aeronave_tipo_icao',
    'aeronave_pmd', 
    'aeronave_pmd_categoria',
    'aeronave_assentos', 
    'aeronave_ano_fabricacao',
    'aeronave_pais_fabricante', 
    'aeronave_pais_registro',
    'aeronave_registro_segmento',
    ], axis=1)

In [6]:
# removendo linhas duplicadas
df_ocorr = df_ocorr.drop_duplicates()

Tratando Tipos de Dados

In [7]:
#definindo uma função para normalizar as strings
def normalizar_string(string):

    string_normalizada = string.strip() # remove espaços em branco no início e no final da string
    string_normalizada = string_normalizada.lower() # transforma todas as letras em minúsculas
    string_normalizada = unidecode.unidecode(string_normalizada)
    string_normalizada = re.sub(r'[^\w\s-]', '', string_normalizada)

    return string_normalizada

In [8]:
# definindo os tipos de dados de cada coluna
df_ocorr["ocorrencia_classificacao"] = df_ocorr["ocorrencia_classificacao"].apply(normalizar_string).astype("category")       
df_ocorr["ocorrencia_latitude"] = df_ocorr["ocorrencia_latitude"].apply(lambda x: ".".join(str(x).split(".")[0:2]).replace("°","")).astype(float)        
df_ocorr["ocorrencia_longitude"] = df_ocorr["ocorrencia_longitude"].apply(lambda x: ".".join(str(x).split(".")[0:2]).replace("°","")).astype(float)         
df_ocorr["ocorrencia_cidade"] = df_ocorr["ocorrencia_cidade"].apply(normalizar_string).astype("category")        
df_ocorr["ocorrencia_uf"] = df_ocorr["ocorrencia_uf"].apply(normalizar_string).astype("category")                        
df_ocorr["ocorrencia_aerodromo"] = df_ocorr["ocorrencia_aerodromo"].apply(lambda x: x.strip().lower()).astype("category")           
df_ocorr["total_recomendacoes"] = df_ocorr["total_recomendacoes"].astype(int)            
df_ocorr["total_aeronaves_envolvidas"] = df_ocorr["total_aeronaves_envolvidas"].astype(int)     
df_ocorr["ocorrencia_saida_pista"] = df_ocorr["ocorrencia_saida_pista"].apply(lambda x: x.strip().lower()).astype("category")         
df_ocorr["ocorrencia_tipo"] = df_ocorr["ocorrencia_tipo"].apply(normalizar_string).astype("category")            
df_ocorr["ocorrencia_tipo_categoria"] = df_ocorr["ocorrencia_tipo_categoria"].apply(normalizar_string).astype("category")      
df_ocorr["taxonomia_tipo_icao"] = df_ocorr["taxonomia_tipo_icao"].apply(lambda x: x.strip()).astype("category")            
df_ocorr["recomendacao_status"] = df_ocorr["recomendacao_status"].apply(lambda x: x.strip().lower()).astype("category")            
df_ocorr["recomendacao_destinatario"] = df_ocorr["recomendacao_destinatario"].apply(lambda x: x.strip().lower()).astype("category")      
df_ocorr["aeronave_tipo_veiculo"] = df_ocorr["aeronave_tipo_veiculo"].apply(normalizar_string).astype("category")          
df_ocorr["aeronave_fabricante"] = df_ocorr["aeronave_fabricante"].apply(normalizar_string).astype("category")            
df_ocorr["aeronave_modelo"] = df_ocorr["aeronave_modelo"].apply(lambda x: x.strip().lower()).astype("category")                
df_ocorr["aeronave_motor_tipo"] = df_ocorr["aeronave_motor_tipo"].apply(normalizar_string).astype("category")            
df_ocorr["aeronave_motor_quantidade"] = df_ocorr["aeronave_motor_quantidade"].apply(normalizar_string).astype("category")    
df_ocorr["aeronave_voo_origem"] = df_ocorr["aeronave_voo_origem"].apply(lambda x: x.strip().lower()).astype(str).astype("category")             
df_ocorr["aeronave_voo_destino"] = df_ocorr["aeronave_voo_destino"].apply(lambda x: x.strip().lower()).astype(str).astype("category")            
df_ocorr["aeronave_fase_operacao"] = df_ocorr["aeronave_fase_operacao"].apply(normalizar_string).astype("category")         
df_ocorr["aeronave_tipo_operacao"] = df_ocorr["aeronave_tipo_operacao"].apply(normalizar_string).astype("category")         
df_ocorr["aeronave_nivel_dano"] = df_ocorr["aeronave_nivel_dano"].apply(normalizar_string).astype("category")            
df_ocorr["aeronave_fatalidades_total"] = df_ocorr["aeronave_fatalidades_total"].astype(int)      
df_ocorr["fator_nome"] = df_ocorr["fator_nome"].apply(normalizar_string).astype("category")                    
df_ocorr["fator_aspecto"] = df_ocorr["fator_aspecto"].apply(normalizar_string).astype("category")                  
df_ocorr["fator_condicionante"] = df_ocorr["fator_condicionante"].apply(normalizar_string).astype("category")            
df_ocorr["fator_area"] = df_ocorr["fator_area"].apply(normalizar_string).astype("category")
# df_ocorr["ocorrencia_dia"] = df_ocorr["ocorrencia_dia"] = pd.to_datetime(df_ocorr["ocorrencia_dia"], format='%d/%m/%Y').dt.date               
# df_ocorr["ocorrencia_hora"] = df_ocorr["ocorrencia_hora"] = pd.to_datetime(df_ocorr["ocorrencia_hora"], format='%H:%M:%S').dt.time               
# df_ocorr["investigacao_aeronave_liberada"] = df_ocorr["investigacao_aeronave_liberada"].apply(lambda x: x.strip().lower()).astype("category") 
# df_ocorr["divulgacao_relatorio_numero"] = df_ocorr["divulgacao_relatorio_numero"].apply(lambda x: x.strip()).astype(str) 
# df_ocorr["divulgacao_dia_publicacao"] = pd.to_datetime(df_ocorr["divulgacao_dia_publicacao"]).dt.date     
# df_ocorr["aeronave_matricula"] = df_ocorr["aeronave_matricula"].apply(lambda x: x.strip().lower()).astype(str)            
# df_ocorr["recomendacao_numero"] = df_ocorr["recomendacao_numero"].apply(lambda x: x.replace(" ","")).astype(str)            
# df_ocorr["recomendacao_dia_assinatura"] = pd.to_datetime(df_ocorr["ocorrencia_dia"], format="YYYY-MM-DD").dt.date    
# df_ocorr["recomendacao_dia_encaminhamento"] = pd.to_datetime(df_ocorr["ocorrencia_dia"], format="YYYY-MM-DD").dt.date
# df_ocorr["recomendacao_dia_feedback"] = pd.to_datetime(df_ocorr["ocorrencia_dia"], format="YYYY-MM-DD").dt.date      
# df_ocorr["recomendacao_conteudo"] = df_ocorr["recomendacao_conteudo"].apply(lambda x: x.strip().lower()).astype(str)          
# df_ocorr["aeronave_tipo_icao"] = df_ocorr["aeronave_tipo_icao"].apply(lambda x: x.strip().lower()).astype("category")             
# df_ocorr["aeronave_pmd"] = df_ocorr["aeronave_pmd"].astype(int)                   
# df_ocorr["aeronave_pmd_categoria"] = df_ocorr["aeronave_pmd_categoria"].astype(int)         
# df_ocorr["aeronave_assentos"] = df_ocorr["aeronave_assentos"].astype(int)              
# df_ocorr["aeronave_ano_fabricacao"] = df_ocorr["aeronave_ano_fabricacao"].astype(int)        
# df_ocorr["aeronave_pais_fabricante"] = df_ocorr["aeronave_pais_fabricante"].apply(normalizar_string).astype("category")       
# df_ocorr["aeronave_pais_registro"] = df_ocorr["aeronave_pais_registro"].apply(lambda x: x.strip().lower()).astype("category")           
# df_ocorr["aeronave_registro_segmento"] = df_ocorr["aeronave_registro_segmento"].apply(normalizar_string).astype("category")     

In [9]:
# unindo dia e hora da ocorrência na coluna ocorrencia_data_hora
df_ocorr["ocorrencia_data_hora"] = df_ocorr["ocorrencia_dia"].astype(str) + " " + df_ocorr["ocorrencia_hora"].astype(str)
df_ocorr = df_ocorr.drop(["ocorrencia_dia", "ocorrencia_hora"], axis=1)
df_ocorr["ocorrencia_data_hora"] = pd.to_datetime(df_ocorr["ocorrencia_data_hora"])
# df_ocorr.head(3)

C:\Users\victo\AppData\Local\Temp\ipykernel_29812\3857574046.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_ocorr["ocorrencia_data_hora"] = pd.to_datetime(df_ocorr["ocorrencia_data_hora"])


In [10]:
# Salvando dados tratados de Ocorrências Aeronáuticas
df_ocorr.to_csv(r'..\data\df_ocorrencias_aeronauticas_tratado.csv', index=False)

Análise da Distribuição de Ocorrências em intervalos de tempo

In [11]:
# Percentural de Ocorrências x Hora[dia]
df_ocorr["ocorrencia_hora"] = df_ocorr["ocorrencia_data_hora"].dt.hour
df_ocorr_ocorr_hora = df_ocorr["ocorrencia_hora"].value_counts().sort_index()
df_ocorr_ocorr_hora = pd.DataFrame(df_ocorr_ocorr_hora)
df_ocorr_ocorr_hora.reset_index(inplace=True)

grf_ocorr_hora = px.line(
    data_frame=df_ocorr_ocorr_hora,
    x=df_ocorr_ocorr_hora["ocorrencia_hora"].apply(lambda x : f"{x}h"),
    y=(df_ocorr_ocorr_hora["count"]/df_ocorr_ocorr_hora["count"].sum())*100,
    title=f"Ocorrências x Hora",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
grf_ocorr_hora.update_layout(
    title_x = 0.5,
    title_y = 0.9,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    # yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=20,  # margem esquerda
        r=20,  # margem direita
        b=20,  # margem inferior
        t=0,  # margem superior
        pad=0  # preenchimento
    ),
)
grf_ocorr_hora.update_traces(
    hovertemplate='%{x} : %{y}% ocorrências',
    fill='tozeroy', 
    # text=(df_ocorr_ocorr_hora["count"] / df_ocorr_ocorr_hora["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    # mode='lines+text', 
    # textposition='top center',
    )
grf_ocorr_hora.update_xaxes(title = "")
grf_ocorr_hora.update_yaxes(title = "Ocorrências [%]")

grf_ocorr_hora = px.line(
    data_frame=df_ocorr_ocorr_hora,
    x=df_ocorr_ocorr_hora["ocorrencia_hora"].apply(lambda x : f"{x}h"),
    y=(df_ocorr_ocorr_hora["count"]/df_ocorr_ocorr_hora["count"].sum())*100,
    title=f"Ocorrências x Hora",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
grf_ocorr_hora.update_layout(
    title_x = 0.5,
    title_y = 0.96,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    # yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=20,  # margem esquerda
        r=20,  # margem direita
        b=20,  # margem inferior
        t=0,  # margem superior
        pad=0  # preenchimento
    ),
)
grf_ocorr_hora.update_traces(
    hovertemplate='%{x} : %{y}% ocorrências',
    fill='tozeroy', 
    text=(df_ocorr_ocorr_hora["count"] / df_ocorr_ocorr_hora["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    mode='lines+text', 
    textposition='top center',
    )
grf_ocorr_hora.update_xaxes(title = "")
grf_ocorr_hora.update_yaxes(visible=False)

# Exibir o histograma
grf_ocorr_hora.show()

In [12]:
# Percentural de Ocorrências x Dia[semana]
df_ocorr["ocorrencia_dia_semana"] = df_ocorr["ocorrencia_data_hora"].dt.day_of_week

def converter_numero_dia_semana(num_dia:int) -> str:
    match num_dia:
        case 0:return "domingo"
        case 1:return "segunda"
        case 2:return "terça"
        case 3:return "quarta"
        case 4:return "quinta"
        case 5:return "sexta"
        case 6:return "sabado"

df_ocorr_ocorr_dia_semana = df_ocorr["ocorrencia_dia_semana"].value_counts().sort_index()
df_ocorr_ocorr_dia_semana = pd.DataFrame(df_ocorr_ocorr_dia_semana)
df_ocorr_ocorr_dia_semana.reset_index(inplace=True)
df_ocorr_ocorr_dia_semana["ocorrencia_dia_semana"] = df_ocorr_ocorr_dia_semana["ocorrencia_dia_semana"].map(converter_numero_dia_semana)

grf_ocorr_dia_semana = px.line(
    data_frame=df_ocorr_ocorr_dia_semana,
    x=df_ocorr_ocorr_dia_semana["ocorrencia_dia_semana"],
    y=(df_ocorr_ocorr_dia_semana["count"]/df_ocorr_ocorr_dia_semana["count"].sum())*100,
    title=f"Ocorrências x Dia[semana]",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
grf_ocorr_dia_semana.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=20,  # margem esquerda
        r=20,  # margem direita
        b=20,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
grf_ocorr_dia_semana.update_traces(
    hovertemplate='%{x} : %{y}% ocorrências',
    fill='tozeroy', 
    text=(df_ocorr_ocorr_dia_semana["count"] / df_ocorr_ocorr_dia_semana["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    mode='lines+text', 
    textposition='top center',
    )
grf_ocorr_dia_semana.update_xaxes(title = "")
grf_ocorr_dia_semana.update_yaxes(visible=False)

# Exibir o histograma
grf_ocorr_dia_semana.show()

In [13]:
# Percentural de Ocorrências x Dia[mês]
df_ocorr["ocorrencia_dia_mes"] = df_ocorr["ocorrencia_data_hora"].dt.day
df_ocorr_ocorr_dia_mes = df_ocorr["ocorrencia_dia_mes"].value_counts().sort_index()
df_ocorr_ocorr_dia_mes = pd.DataFrame(df_ocorr_ocorr_dia_mes)
df_ocorr_ocorr_dia_mes.reset_index(inplace=True)

grf_ocorr_dia_mes = px.line(
    data_frame=df_ocorr_ocorr_dia_mes,
    x=df_ocorr_ocorr_dia_mes["ocorrencia_dia_mes"],
    y=(df_ocorr_ocorr_dia_mes["count"]/df_ocorr_ocorr_dia_mes["count"].sum())*100,
    title=f"Ocorrências x Dia[mês]",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
grf_ocorr_dia_mes.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=20,  # margem esquerda
        r=20,  # margem direita
        b=20,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
grf_ocorr_dia_mes.update_traces(
    hovertemplate='Dia %{x} : %{y} ocorrências',
    fill='tozeroy', 
    text=(df_ocorr_ocorr_dia_mes["count"] / df_ocorr_ocorr_dia_mes["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    mode='lines+text', 
    textposition='top center',
    )
grf_ocorr_dia_mes.update_xaxes(title = "")
grf_ocorr_dia_mes.update_yaxes(visible=False)

# Exibir o histograma
grf_ocorr_dia_mes.show()

In [14]:
# Percentural de Ocorrências x Mês
df_ocorr["ocorrencia_mes"] = df_ocorr["ocorrencia_data_hora"].dt.month

def converter_numero_nome_mes(num_mes:int) -> str:
    match num_mes:
        case 1:return "janeiro"
        case 2:return "fevereiro"
        case 3:return "marco"
        case 4:return "abril"
        case 5:return "maio"
        case 6:return "junho"
        case 7:return "julho"
        case 8:return "agosto"
        case 9:return "setembro"
        case 10:return "outubro"
        case 11:return "novembro"
        case 12:return "dezembro"

df_ocorr_ocorr_mes = df_ocorr["ocorrencia_mes"].value_counts().sort_index()
df_ocorr_ocorr_mes = pd.DataFrame(df_ocorr_ocorr_mes)
df_ocorr_ocorr_mes.reset_index(inplace=True)
df_ocorr_ocorr_mes["ocorrencia_mes"] = df_ocorr_ocorr_mes["ocorrencia_mes"].map(converter_numero_nome_mes)

grf_ocorr_mes = px.line(
    data_frame=df_ocorr_ocorr_mes,
    x=df_ocorr_ocorr_mes["ocorrencia_mes"],
    y=(df_ocorr_ocorr_mes["count"]/df_ocorr_ocorr_mes["count"].sum())*100,
    title=f"Ocorrências[%] x Mês",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
grf_ocorr_mes.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=25,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
    # xaxis_range=['janeiro','dezembro'],
)
grf_ocorr_mes.update_traces(
    hovertemplate='%{x} : %{y}% ocorrências',
    fill='tozeroy', 
    text=(df_ocorr_ocorr_mes["count"] / df_ocorr_ocorr_mes["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    mode='lines+text', 
    textposition='top center',
    )
grf_ocorr_mes.update_xaxes(title = "", tickangle=45)
grf_ocorr_mes.update_yaxes(visible=False)

# Exibir o histograma
grf_ocorr_mes.show()

In [15]:
# Percentural de Ocorrências x Ano
df_ocorr["ocorrencia_ano"] = df_ocorr["ocorrencia_data_hora"].dt.year
df_ocorr_ocorr_ano = df_ocorr["ocorrencia_ano"].value_counts().sort_index()
df_ocorr_ocorr_ano = pd.DataFrame(df_ocorr_ocorr_ano)
df_ocorr_ocorr_ano.reset_index(inplace=True)

grf_ocorr_ano = px.line(
    data_frame=df_ocorr_ocorr_ano,
    x=df_ocorr_ocorr_ano["ocorrencia_ano"],
    y=(df_ocorr_ocorr_ano["count"]/df_ocorr_ocorr_ano["count"].sum())*100,
    title=f"Ocorrências[%] x Ano",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
grf_ocorr_ano.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=40,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
    xaxis_range=[2012, 2022],
)
grf_ocorr_ano.update_traces(
    hovertemplate='%{x} : %{y}% ocorrências', 
    fill='tozeroy', 
    text=(df_ocorr_ocorr_ano["count"] / df_ocorr_ocorr_ano["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    mode='lines+text', 
    textposition='top center',
    )
grf_ocorr_ano.update_xaxes(title = "", tickangle=45)
grf_ocorr_ano.update_yaxes(visible=False)

# Exibir o histograma
grf_ocorr_ano.show()

Análise dos tipos de ocorrência

In [16]:
# ocorrencia_tipo
df_ocorr_tipo_ocorr = df_ocorr["ocorrencia_tipo"].value_counts().sort_values(ascending=False)
df_ocorr_tipo_ocorr = pd.DataFrame(df_ocorr_tipo_ocorr)
df_ocorr_tipo_ocorr = df_ocorr_tipo_ocorr.head(15)
df_ocorr_tipo_ocorr.reset_index(inplace=True)

hist_tipo_ocorr = px.bar(
    data_frame=df_ocorr_tipo_ocorr,
    x=df_ocorr_tipo_ocorr["ocorrencia_tipo"],
    y=(df_ocorr_tipo_ocorr["count"]/df_ocorr_tipo_ocorr["count"].sum())*100,
    title=f"Ocorrências[%] x Tipo de Ocorrência",
    width=600,
    height=600,
    color_discrete_sequence=["#2570DC"],
    )
hist_tipo_ocorr.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_tipo_ocorr.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_tipo_ocorr["count"] / df_ocorr_tipo_ocorr["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_tipo_ocorr.update_xaxes(title = "", tickangle=45)
hist_tipo_ocorr.update_yaxes(visible=False)
hist_tipo_ocorr.show()

Análise dos modelos e marcas de aeronaves

In [17]:
# Ocorrências[%] x Tipo de Aeronave
df_ocorr_tipo_veic = df_ocorr["aeronave_tipo_veiculo"].value_counts().sort_values(ascending=False)
df_ocorr_tipo_veic = pd.DataFrame(df_ocorr_tipo_veic)
df_ocorr_tipo_veic = df_ocorr_tipo_veic.head(15)
df_ocorr_tipo_veic.reset_index(inplace=True)

hist_ocorr_tipo_veic = px.bar(
    data_frame=df_ocorr_tipo_veic,
    x=df_ocorr_tipo_veic["aeronave_tipo_veiculo"],
    y=(df_ocorr_tipo_veic["count"]/df_ocorr_tipo_veic["count"].sum())*100,
    title=f"Ocorrências[%] x Tipo de Aeronave",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
hist_ocorr_tipo_veic.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_ocorr_tipo_veic.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_tipo_veic["count"] / df_ocorr_tipo_veic["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_ocorr_tipo_veic.update_xaxes(title = "", tickangle=45)
hist_ocorr_tipo_veic.update_yaxes(visible=False)
hist_ocorr_tipo_veic.show()

In [18]:
# Ocorrências[%] x Fabricante
df_ocorr_fabr = df_ocorr["aeronave_fabricante"].value_counts().sort_values(ascending=False)
df_ocorr_fabr = pd.DataFrame(df_ocorr_fabr)
df_ocorr_fabr = df_ocorr_fabr.head(15)
df_ocorr_fabr.reset_index(inplace=True)

hist_ocorr_fabr = px.bar(
    data_frame=df_ocorr_fabr,
    x=df_ocorr_fabr["aeronave_fabricante"],
    y=(df_ocorr_fabr["count"]/df_ocorr_fabr["count"].sum())*100,
    title=f"Ocorrências[%] x Fabricante",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
hist_ocorr_fabr.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_ocorr_fabr.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_fabr["count"] / df_ocorr_fabr["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_ocorr_fabr.update_xaxes(title = "", tickangle=45)
hist_ocorr_fabr.update_yaxes(visible=False)
hist_ocorr_fabr.show()

In [19]:
#definindo uma função para normalizar as strings
def normalizar_string(string):

    string_normalizada = string.strip() # remove espaços em branco no início e no final da string
    string_normalizada = string_normalizada.lower() # transforma todas as letras em minúsculas
    string_normalizada = unidecode.unidecode(string_normalizada)
    string_normalizada = re.sub(r'[^\w\s-]', '', string_normalizada)

    return string_normalizada

In [20]:
# Ocorrências[%] x Modelo de Aeronave
df_ocorr_mod = df_ocorr["aeronave_modelo"].value_counts().sort_values(ascending=False)
df_ocorr_mod = pd.DataFrame(df_ocorr_mod)
df_ocorr_mod = df_ocorr_mod.head(15)
df_ocorr_mod.reset_index(inplace=True)

hist_ocorr_mod = px.bar(
    data_frame=df_ocorr_mod,
    x=df_ocorr_mod["aeronave_modelo"],
    y=(df_ocorr_mod["count"]/df_ocorr_mod["count"].sum())*100,
    title=f"Ocorrências[%] x Modelo de Aeronave",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
hist_ocorr_mod.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_ocorr_mod.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_mod["count"] / df_ocorr_mod["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_ocorr_mod.update_xaxes(title = "", tickangle=45)
hist_ocorr_mod.update_yaxes(visible=False)
hist_ocorr_mod.show()

Análise dos Fatores Críticos

In [21]:
# Ocorrências[%] x Tipo de Causa
df_ocorr_fat_area = df_ocorr["fator_area"].value_counts().sort_values(ascending=False)
df_ocorr_fat_area = pd.DataFrame(df_ocorr_fat_area)
df_ocorr_fat_area.reset_index(inplace=True)

hist_ocorr_fat_area = px.bar(
    data_frame=df_ocorr_fat_area,
    x=df_ocorr_fat_area["fator_area"],
    y=(df_ocorr_fat_area["count"]/df_ocorr_fat_area["count"].sum())*100,
    title=f"Ocorrências[%] x Tipo de Causa",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
hist_ocorr_fat_area.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_ocorr_fat_area.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_fat_area["count"] / df_ocorr_fat_area["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_ocorr_fat_area.update_xaxes(title = "", tickangle=45)
hist_ocorr_fat_area.update_yaxes(visible=False)
hist_ocorr_fat_area.show()

In [22]:
# Ocorrências[%] x Aspecto da Causa
df_ocorr_fat_asp = df_ocorr["fator_aspecto"].value_counts().sort_values(ascending=False)
df_ocorr_fat_asp = pd.DataFrame(df_ocorr_fat_asp)
df_ocorr_fat_asp = df_ocorr_fat_asp.head(6)
df_ocorr_fat_asp.reset_index(inplace=True)

hist_ocorr_fat_asp = px.bar(
    data_frame=df_ocorr_fat_asp,
    x=df_ocorr_fat_asp["fator_aspecto"],
    y=(df_ocorr_fat_asp["count"]/df_ocorr_fat_asp["count"].sum())*100,
    title=f"Ocorrências[%] x Aspecto da Causa",
    width=600,
    height=600,
    color_discrete_sequence=["#2570DC"],
    )
hist_ocorr_fat_asp.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_ocorr_fat_asp.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_fat_asp["count"] / df_ocorr_fat_asp["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_ocorr_fat_asp.update_xaxes(title = "", tickangle=45)
hist_ocorr_fat_asp.update_yaxes(visible=False)
hist_ocorr_fat_asp.show()


In [23]:
# Ocorrências[%] x Fator Condicionante
df_ocorr_fat_cond = df_ocorr["fator_condicionante"].value_counts().sort_values(ascending=False)
df_ocorr_fat_cond = pd.DataFrame(df_ocorr_fat_cond)
df_ocorr_fat_cond.reset_index(inplace=True)

hist_ocorr_fat_cond = px.bar(
    data_frame=df_ocorr_fat_cond,
    x=df_ocorr_fat_cond["fator_condicionante"],
    y=(df_ocorr_fat_cond["count"]/df_ocorr_fat_cond["count"].sum())*100,
    title=f"Ocorrências[%] x Fator Condicionante",
    width=600,
    height=550,
    color_discrete_sequence=["#2570DC"],
    )
hist_ocorr_fat_cond.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_ocorr_fat_cond.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_fat_cond["count"] / df_ocorr_fat_cond["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_ocorr_fat_cond.update_xaxes(title = "", tickangle=45)
hist_ocorr_fat_cond.update_yaxes(visible=False)
hist_ocorr_fat_cond.show()


In [24]:
# Ocorrências[%] x Fator
df_ocorr_fator = df_ocorr["fator_nome"].value_counts().sort_values(ascending=False)
df_ocorr_fator = pd.DataFrame(df_ocorr_fator)
df_ocorr_fator = df_ocorr_fator.head(15)
df_ocorr_fator.reset_index(inplace=True)

hist_ocorr_fator = px.bar(
    data_frame=df_ocorr_fator,
    x=df_ocorr_fator["fator_nome"],
    y=(df_ocorr_fator["count"]/df_ocorr_fator["count"].sum())*100,
    title=f"Ocorrências[%] x Fator",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
hist_ocorr_fator.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_ocorr_fator.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_fator["count"] / df_ocorr_fator["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_ocorr_fator.update_xaxes(title = "", tickangle=45)
hist_ocorr_fator.update_yaxes(visible=False)
hist_ocorr_fator.show()

Distribuição geográfica das ocorrências

In [25]:
# obtendo a contagem de ocorrências por coordenada
df_ocorr_ocorr_localizacao = df_ocorr[["ocorrencia_latitude", "ocorrencia_longitude", "ocorrencia_uf", "ocorrencia_cidade"]]
df_ocorr_ocorr_localizacao["localizacao"] = df_ocorr_ocorr_localizacao.apply(lambda linha : (linha["ocorrencia_latitude"], linha["ocorrencia_longitude"]),axis=1)
df_ocorr_ocorr_localizacao = pd.DataFrame(df_ocorr_ocorr_localizacao.value_counts())
df_ocorr_ocorr_localizacao.reset_index(inplace=True)

# removendo registros com localizações fora da região analisada
df_ocorr_ocorr_localizacao = df_ocorr_ocorr_localizacao.drop(df_ocorr_ocorr_localizacao[df_ocorr_ocorr_localizacao["ocorrencia_latitude"] > 5.7].index, axis=0)
df_ocorr_ocorr_localizacao = df_ocorr_ocorr_localizacao.drop(df_ocorr_ocorr_localizacao[df_ocorr_ocorr_localizacao["ocorrencia_longitude"] > -32.7].index, axis=0)
df_ocorr_ocorr_localizacao = df_ocorr_ocorr_localizacao.drop(df_ocorr_ocorr_localizacao[df_ocorr_ocorr_localizacao["localizacao"] == (0.0, 0.0)].index, axis=0)

# gerando mapa base
mapa = folium.Map(
    width="100%",
    height="100%",
    location=[-15.7801, -47.9292], 
    zoom_start=5,
    tiles='cartodbpositron',)

# adicionando exibição de coordenadas
MousePosition().add_to(mapa)

# gerando uma lista com a contagem de ocorrências por par de coordenadas
coordenadas = df_ocorr_ocorr_localizacao[["ocorrencia_latitude", "ocorrencia_longitude", "count"]].values.tolist()

# Criando tooltips com as informações
for i in range(0, len(df_ocorr_ocorr_localizacao)):
    folium.Circle(
        location = [df_ocorr_ocorr_localizacao.iloc[i]['ocorrencia_latitude'], df_ocorr_ocorr_localizacao.iloc[i]['ocorrencia_longitude']],
        color = '#0000cc',
        fill = '#A4D3EE',
        tooltip='<li><b>Estado: </b>' + str(df_ocorr_ocorr_localizacao.iloc[i]["ocorrencia_uf"]) +
                '<li><b>Município: </b>' + str(df_ocorr_ocorr_localizacao.iloc[i]["ocorrencia_cidade"]) +
                '<li><b>Ocorrências Aeronáuticas: </b>' + str(df_ocorr_ocorr_localizacao.iloc[i]['count']),
        radius=10
    ).add_to(mapa)

# gerando mapar de calor
HeatMap(coordenadas, radius=15).add_to(mapa)

# visualizando mapa de Calor
mapa

C:\Users\victo\AppData\Local\Temp\ipykernel_29812\405705810.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Análise das Classificações dos Acidentes

In [26]:
# Ocorrências[%] x Classificação
df_ocorr_clas = df_ocorr["ocorrencia_classificacao"].value_counts().sort_values(ascending=False)
df_ocorr_clas = pd.DataFrame(df_ocorr_clas)
df_ocorr_clas = df_ocorr_clas.head(15)
df_ocorr_clas.reset_index(inplace=True)

hist_ocorr_clas = px.bar(
    data_frame=df_ocorr_clas,
    x=df_ocorr_clas["ocorrencia_classificacao"],
    y=(df_ocorr_clas["count"]/df_ocorr_clas["count"].sum())*100,
    title=f"Ocorrências[%] x Classificação",
    width=600,
    height=500,
    color_discrete_sequence=["#2570DC"],
    )
hist_ocorr_clas.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
hist_ocorr_clas.update_traces(
    hovertemplate='%{x} : %{y}%',
    text=(df_ocorr_clas["count"] / df_ocorr_clas["count"].sum() * 100).round(0).astype(int).astype(str) + "%",
    textposition='outside',
    )
hist_ocorr_clas.update_xaxes(title = "", tickangle=45)
hist_ocorr_clas.update_yaxes(visible=False)
hist_ocorr_clas.show()

Correlação entre Danos e Fatalidades

In [27]:
# Analisando a Correlação entre Danos e Fatalidades
df_ocorr_danos_fatal = df_ocorr.loc[df_ocorr["ocorrencia_classificacao"] == 'acidente',:]
df_ocorr_danos_fatal['aeronave_nivel_dano'] = df_ocorr_danos_fatal['aeronave_nivel_dano'].astype(str)
df_ocorr_danos_fatal = df_ocorr_danos_fatal[df_ocorr_danos_fatal['aeronave_nivel_dano'] != 'desconhecido']
df_ocorr_danos_fatal = df_ocorr_danos_fatal.loc[:,['aeronave_nivel_dano', 'aeronave_fatalidades_total']]
df_ocorr_danos_fatal = df_ocorr_danos_fatal.groupby(by=['aeronave_nivel_dano', 'aeronave_fatalidades_total']).value_counts()
df_ocorr_danos_fatal = df_ocorr_danos_fatal.reset_index()
df_ocorr_danos_fatal["aeronave_fatalidades_total"] = df_ocorr_danos_fatal["aeronave_fatalidades_total"].astype(str)

grf_danos_fatal = px.bar(
    data_frame=df_ocorr_danos_fatal,
    x=df_ocorr_danos_fatal["aeronave_fatalidades_total"],
    y=df_ocorr_danos_fatal["count"],
    color=df_ocorr_danos_fatal["aeronave_nivel_dano"],
    title= 'Danos e Fatalidades',
    barmode='group',
    height=500,
    width=800,
    )
grf_danos_fatal.update_layout(
    title_x = 0.5,
    title_y = 0.95,
    plot_bgcolor='#BDE4F4',  # Cor de fundo do gráfico
    paper_bgcolor='#BDE4F4',  # Cor de fundo do papel
    xaxis_showgrid=False,  # Remover linhas de grade do eixo x
    yaxis_showgrid=False,  # Remover linhas de grade do eixo y
    bargap = 0.1,
    xaxis=dict(
        tickmode='linear',  # Modo linear
        tick0=0,  # Posição inicial dos ticks
        dtick=1  # Passo entre os ticks
    ),
    margin=dict(
        l=10,  # margem esquerda
        r=10,  # margem direita
        b=0,  # margem inferior
        t=35,  # margem superior
        pad=0  # preenchimento
    ),
)
grf_danos_fatal.update_traces(
    hovertemplate='%{y}',
    text=df_ocorr_danos_fatal["count"],
    textposition='outside',
    textangle=270,
    )
grf_danos_fatal.update_xaxes(title = "")
grf_danos_fatal.update_yaxes(visible=False)

grf_danos_fatal.show()

C:\Users\victo\AppData\Local\Temp\ipykernel_29812\44564398.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Análise de Causa Raiz

In [28]:
# • Aplicar métodos estatísticos ou algoritmos de aprendizado de máquina para identificar
# fatores que são mais frequentemente associados a diferentes tipos ou gravidades de
# acidentes. Isso pode ajudar a identificar áreas críticas para melhorias de segurança.
# • Análise Temporal e de Séries Temporais:
# • Analisar como a frequência e a gravidade dos acidentes mudaram ao longo do tempo e se
# há padrões sazonais ou relacionados a eventos específicos.
# • Detecção de Anomalias:
# • Usar técnicas de detecção de anomalias para identificar acidentes que são atípicos em
# termos de suas características ou consequências, o que pode sinalizar áreas para
# investigação mais aprofundada.
# • Criação de Dashboards Interativos:
# • Desenvolver dashboards interativos que permitam aos usuários explorar os dados e
# descobertas de maneira intuitiva, filtrando por diferentes variáveis, visualizando
# tendências ao longo do tempo e examinando detalhes de acidentes específicos.


In [29]:
df_ocorr.describe()

,ocorrencia_latitude,ocorrencia_longitude,total_recomendacoes,total_aeronaves_envolvidas,aeronave_fatalidades_total,ocorrencia_data_hora,ocorrencia_hora,ocorrencia_dia_semana,ocorrencia_dia_mes,ocorrencia_mes,ocorrencia_ano
count,3879.000000,3879.000000,3879.000000,3879.000000,3879.000000,3879,3879.000000,3879.000000,3879.000000,3879.000000,3879.000000
mean,-26.579722,-70.671641,2.798144,1.072184,0.803042,2017-04-17 14:36:51.724671488,14.910028,3.066254,15.555556,6.182521,2016.821346
min,-256.002000,-514.402000,1.000000,1.000000,0.000000,2013-01-03 16:00:00,0.000000,0.000000,1.000000,1.000000,2013.000000
25%,-23.868611,-53.465000,1.000000,1.000000,0.000000,2015-02-07 11:00:00,12.000000,2.000000,8.000000,3.000000,2015.000000
50%,-20.363333,-48.553056,2.000000,1.000000,0.000000,2017-07-20 11:00:00,15.000000,3.000000,15.000000,6.000000,2017.000000
75%,-8.126389,-44.028194,3.000000,1.000000,1.000000,2019-03-08 11:15:00,19.000000,5.000000,24.000000,9.000000,2019.000000
max,19.813889,63.902778,13.000000,2.000000,10.000000,2022-04-14 17:45:00,23.000000,6.000000,31.000000,12.000000,2022.000000
std,43.032454,103.467350,2.374816,0.258825,1.788128,NaN,4.519558,1.919202,8.945249,3.573251,2.522268
